## <center style="color:red">**News Classifier**</center>

### <center>**ML Pipeline with Sentence Transformers, ChromaDB & Airflow**</center>

Ce projet vise à construire un pipeline complet de classification de textes permettant de catégoriser automatiquement des articles d’actualité dans 4 classes :

- World
- Sports
- Business
- Sci/Tech

Le pipeline utilise des techniques NLP avancées, des embeddings vectoriels, une base de données vectorielle et une orchestration via Apache Airflow.

<br>

### <span style="color:green">**Embeddings & Vectorisation :**</span>

#### <span style="color:orange">**1. Charger les Données :**</span>

In [1]:
import pandas as pd

df_train = pd.read_csv("../data/processed/train.csv")

print("Données Chargées avec Succès !")

df_train.head()

Données Chargées avec Succès !


,text,clean_text,label
0,wall st. bears claw back into the black (reute...,wall bears claw back black reuters reuters wal...,2
1,carlyle looks toward commercial aerospace (reu...,carlyle looks toward commercial aerospace reut...,2
2,oil and economy cloud stocks' outlook (reuters...,oil economy cloud stocks outlook reuters reute...,2
3,iraq halts oil exports from main southern pipe...,iraq halts oil exports main southern pipeline ...,2
4,"oil prices soar to all-time record, posing new...",oil prices soar record posing new menace us ec...,2


In [2]:
df_test = pd.read_csv("../data/processed/test.csv")

print("Données Chargées avec Succès !")

df_test.head()

Données Chargées avec Succès !


,text,clean_text,label
0,fears for t n pension after talks unions repre...,fears n pension talks unions representing work...,2
1,the race is on: second private team sets launc...,race second private team sets launch date huma...,3
2,ky. company wins grant to study peptides (ap) ...,company wins grant study peptides ap ap compan...,3
3,prediction unit helps forecast wildfires (ap) ...,prediction unit helps forecast wildfires ap ap...,3
4,calif. aims to limit farm-related smog (ap) ap...,aims limit smog ap ap southern california agen...,3


#### <span style="color:orange">**2. Encoder les Textes d’Entraînement et Test :**</span>

##### **2.1. Sentence Transformer :**

Un `Sentence Transformer` est un modèle de langage basé sur l’architecture **Transformer**, spécialement conçu pour produire des représentations vectorielles (**embeddings**) de phrases plutôt que de simples mots. 

Contrairement aux modèles classiques qui analysent chaque mot séparément, un `Sentence Transformer` capture le sens global d’une phrase, sa structure et ses relations sémantiques. Cela permet de :

- Comparer facilement deux phrases (similarité).

- Regrouper des textes proches.

- Faire de la recherche intelligente, du clustering, ou encore d’alimenter des modèles de classification avec des représentations beaucoup plus riches et précises. 

En pratique, il est largement utilisé dans des tâches de NLP comme la détection de doublons, le question-réponse, le résumé, la classification de textes avec peu d’exemples (few-shot), et bien plus.

Pour installer Sentence Transformers, voici la commande officielle :

```bash
pip install sentence-transformers
```

In [3]:
import sentence_transformers

print("Version : ", sentence_transformers.__version__)

c:\Users\abdel\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Version :  5.1.2


##### **2.2. Charger le Modèle `paraphrase-multilingual-MiniLM-L12-v2` :**

Le `paraphrase-multilingual-MiniLM-L12-v2` est un modèle de la famille **Sentence Transformers**, entraîné pour générer des embeddings de phrases capables de mesurer la similarité sémantique entre textes. 

Ce modèle est multilingue, couvrant plus de 50 langues, ce qui lui permet de comprendre et comparer des phrases écrites dans différentes langues tout en gardant un espace vectoriel commun. 

Il est basé sur **MiniLM**, une architecture Transformer légère et rapide, composée ici de 12 couches, offrant un excellent compromis entre performance, vitesse et taille du modèle. 

En pratique, il est utilisé pour des tâches comme la détection de paraphrases, la recherche sémantique, le clustering de textes ou la classification, surtout lorsque l’on travaille sur des données multilingues.

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

##### **2.3. Encoder les Textes :**

In [5]:
texts_train = df_train["text"].tolist()
texts_test = df_test["text"].tolist()

In [6]:
embeddings_train = model.encode(texts_train, normalize_embeddings=True, show_progress_bar=True)

embeddings_train

Batches: 100%|██████████| 3750/3750 [43:00<00:00,  1.45it/s] 


array([[ 0.08452066, -0.01008103,  0.06637681, ..., -0.06347683,
         0.06138301,  0.06853291],
       [ 0.05757697, -0.04762953, -0.05892563, ..., -0.05354639,
         0.10166489,  0.07635632],
       [ 0.00769315, -0.05420009,  0.0099926 , ..., -0.07634497,
        -0.02036024,  0.05035335],
       ...,
       [-0.00733379,  0.01531467, -0.03676148, ...,  0.0200584 ,
         0.02946613,  0.04843622],
       [-0.04644714, -0.00755117, -0.00477977, ..., -0.02791102,
        -0.03229308,  0.02573509],
       [ 0.00502691, -0.02269462,  0.00718506, ...,  0.00981893,
        -0.01004013, -0.04893992]], shape=(120000, 384), dtype=float32)

In [7]:
embeddings_test = model.encode(texts_test, normalize_embeddings=True, show_progress_bar=True)

embeddings_test

Batches: 100%|██████████| 238/238 [02:40<00:00,  1.49it/s]


array([[-0.01411231,  0.15086146, -0.02255305, ..., -0.01153259,
         0.02307641,  0.03963377],
       [-0.01088149,  0.06553683, -0.02277297, ..., -0.03347767,
        -0.13197821,  0.0141266 ],
       [-0.07935794, -0.00152846, -0.05285733, ...,  0.0270975 ,
         0.06568895,  0.05936584],
       ...,
       [ 0.02158561, -0.00492702,  0.05524898, ..., -0.00168797,
         0.05362756, -0.03379632],
       [-0.02377791,  0.00538047,  0.03565134, ..., -0.09892855,
         0.03224505,  0.05978754],
       [ 0.03612331, -0.03080175,  0.03020013, ..., -0.10219812,
         0.00145259, -0.05525691]], shape=(7600, 384), dtype=float32)

#### <span style="color:orange">**3. Sauvegarder les Datasets après Embedding :**</span>

In [ ]:
df_train["text_embedding"] = embeddings_train.tolist()

df_train["id"] = "train_art_" + df_train.index.astype(str)

df_train.to_pickle("../metadatas/train.pkl")

print("Train Dataset Sauvegardé avec Succès !")

Train Dataset Sauvegardé avec Succès !


In [ ]:
df_test["text_embedding"] = embeddings_test.tolist()

df_test["id"] = "test_art_" + df_test.index.astype(str)

df_test.to_pickle("../metadatas/test.pkl")

print("Test Dataset Sauvegardé avec Succès !")

Test Dataset Sauvegardé avec Succès !
